<a href="https://colab.research.google.com/github/Tech-Jonas/Angewandte-Programmierung/blob/main/Qwen3-4B_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# libs
!pip install -q transformers datasets peft accelerate

import json
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig

# Model
model_name = "/content/drive/MyDrive/Colab Notebooks/Qwen3-4B"

# train
json_path = "/content/drive/MyDrive/Colab Notebooks/12B_trainingdata.json"

# tokenzizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Modell
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

# chat Template, /no_think
def format_entry_chat(entry):
    qtext = entry["body"].strip()
    qtype = entry.get("type", "factoid").lower()
    ctx = "\n".join(s["text"].strip() for s in entry.get("snippets", []))

    results = []

    # ausformulierte, ganze antwort
    ideal_ans = entry.get("ideal_answer", "").strip()
    if ideal_ans:
        if qtype == "yesno":
            user_msg = f"Question: {qtext}\nContext:\n{ctx}\nProvide one-sentence ideal answer in English starting with 'Yes,' or 'No,'."
        else:
            user_msg = f"Question: {qtext}\nContext:\n{ctx}\nProvide an ideal answer in English (one paragraph, max 200 words, full sentences)."
        messages = [
            {"role": "system", "content": "/no_think"},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": ideal_ans}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
            enable_thinking=False
        )
        results.append({"text": text})

    # kurze, knappe Antwort
    exact = entry.get("exact_answer", "")
    exact_ans = ", ".join(exact) if isinstance(exact, list) else exact.strip()
    if exact_ans:
        if qtype == "yesno":
            user_msg = f"Question: {qtext}\nContext:\n{ctx}\nAnswer only 'yes' or 'no', in English, no extras."
        elif qtype == "factoid":
            user_msg = f"Question: {qtext}\nContext:\n{ctx}\nProvide up to 5 keywords, comma-separated, in English, no commentary."
        elif qtype == "list":
            user_msg = f"Question: {qtext}\nContext:\n{ctx}\nProvide a comma-separated list of relevant items, in English, no filler words."
        else:
            user_msg = f"Question: {qtext}\nContext:\n{ctx}\nProvide a brief answer in English."
        messages = [
            {"role": "system", "content": "/no_think"},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": exact_ans}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
            enable_thinking=False
        )
        results.append({"text": text})

    return results

# Train laden
with open(json_path, "r", encoding="utf-8") as f:
    raw_data = json.load(f)["questions"]

formatted = []
for entry in raw_data:
    formatted.extend(format_entry_chat(entry))

dataset = DatasetDict({
    "train": Dataset.from_list(formatted)
})

# tokenisieren
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=2048
    )

tokenized = dataset.map(tokenize, batched=True)

# Start LoRA
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# Training
training_args = TrainingArguments(
    output_dir="./qwen3-4b-lora-nothink",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    evaluation_strategy="no",
    save_strategy="epoch",
    save_total_limit=2,
    logging_steps=25,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized["train"],
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

trainer.train()

# save
model.save_pretrained("./qwen3-4b-lora-nothink")
tokenizer.save_pretrained("./qwen3-4b-lora-nothink")

print("✅ Training abgeschlossen – Modell unter ./qwen3-4b-lora-nothink gespeichert.")